In [4]:
import os
import math
import time
from queue import Queue
from collections import deque

import numpy as np

In [5]:
class Point:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

    def __call__(self) -> tuple:
        return (self.x, self.y, self.z)

    def __str__(self) -> str:
        return str((self.x, self.y, self.z))

    def __eq__(self, other):
        return self.__dict__ == other.__dict__


class Edge:
    def __init__(self, p1, p2):
        self.p1 = p1
        self.p2 = p2

    def __str__(self):
        return "{} {}".format(str(self.p1), str(self.p2))

    def __eq__(self, other):
        if self.p1 == other.p1 or self.p1 == other.p2:
            if self.p2 == other.p1 or self.p2 == other.p2:
                return True
        return False


class Face:
    def __init__(self, p1, p2, p3):
        self.p1 = p1
        self.p2 = p2
        self.p3 = p3

        self.e1 = Edge(p1, p2)
        self.e2 = Edge(p2, p3)
        self.e3 = Edge(p3, p1)

    def __str__(self):
        return "{} {} {}".format(str(self.p1), str(self.p2), str(self.p3))

    def __eq__(self, other):
        pass

In [20]:
def cross(a, b):
    return Point(a.y*b.z-a.z*b.y, a.z*b.x-a.x*b.z, a.x*b.y-a.y*b.x)

def dot(a, b):
    return (a.x*b.x)+(a.y*b.y)+(a.z*b.z)

def norma(a):
    return math.sqrt(a.x*a.x + a.y*a.y + a.z*a.z)

def angle(a, b):
    normaa = norma(a)
    normab = norma(b)
    if norma(a)==0 or norma(b) == 0:
        return -999999999
    return dot(a, b) / (norma(a)*norma(b))

def sub(a, b):
    return Point(a.x-b.x, a.y-b.y, a.z-b.z)

def normal(face):
    v1 = sub(face.p2, face.p1)    
    v2 = sub(face.p3, face.p1)
    return cross(v1,v2)

In [6]:
def read_vertices_from_obj(objPath):
    points = []
    with open(objPath, 'r') as f:
        for line in f:
            try:
                line = line.replace('v', '')
            except:
                #                 print("Problema na linha")
                continue
            line_ = line.split()
            try:
                points.append(
                    Point(float(line_[0]), float(line_[1]), float(line_[2])))
            except:
                #                 print("Problema no ponto")
                continue
    return points


def write_faces(F, P, filename=False):
    i1 = 0
    i2 = 0
    i3 = 0
    if not filename:
        filename = os.path.join(os.getcwd(), 'obj', 'faces.obj')
    with open(filename, 'w') as f:
        for point in P:
            f.write("v {} {} {}".format(point.x, point.y, point.z))
            f.write('\n')
        f.write('\n')
        for i in range(len(F)):
            for j in range(len(P)):
                if F[i].p1.x == P[j].x and F[i].p1.y == P[j].y and F[i].p1.z == P[j].z:
                    i1 = j
                if F[i].p2.x == P[j].x and F[i].p2.y == P[j].y and F[i].p2.z == P[j].z:
                    i2 = j
                if F[i].p3.x == P[j].x and F[i].p3.y == P[j].y and F[i].p3.z == P[j].z:
                    i3 = j
            f.writelines("f {} {} {}".format(i1+1, i2+1, i3+1))
            f.write('\n')


def print_faces(F, P):
    i1 = 0
    i2 = 0
    i3 = 0
    for i in range(len(F)):
        for j in range(len(P)):
            if F[i].p1.x == P[j].x and F[i].p1.y == P[j].y and F[i].p1.z == P[j].z:
                i1 = j
            if F[i].p2.x == P[j].x and F[i].p2.y == P[j].y and F[i].p2.z == P[j].z:
                i2 = j
            if F[i].p3.x == P[j].x and F[i].p3.y == P[j].y and F[i].p3.z == P[j].z:
                i3 = j

        print("f {} {} {}".format(i1+1, i2+1, i3+1))


def write_points_on_hull(H, P, filename=False):
    points_to_write = []
    for face in H:
        if face.p1 not in points_to_write:
            points_to_write.append(face.p1)
        if face.p2 not in points_to_write:
            points_to_write.append(face.p2)
        if face.p3 not in points_to_write:
            points_to_write.append(face.p3)
    if not filename:
        filename = os.path.join(os.getcwd(), 'obj', 'hull.obj')
    with open(filename, 'w') as f:
        for p in points_to_write:
            f.write("v {} {} {}".format(p.x, p.y, p.z))
            f.write('\n')

In [7]:
vertices = read_vertices_from_obj('obj/hullTest.obj')

In [8]:
e1 = Edge(vertices[0], vertices[1])
e2 = Edge(vertices[1], vertices[2])
e3 = Edge(vertices[2], vertices[3])
e4 = Edge(vertices[3], vertices[0])

In [38]:
q = deque()

q.append(e1)
q.append(e2)
q.append(e3)
q.append(e4)
q

deque([<__main__.Edge at 0x1729609ae20>,
       <__main__.Edge at 0x1729609ae50>])

In [39]:
while len(q) > 0:
    max_angle = -float("inf")
    e = q.popleft()
    print("First edge:", e)
    print("P1:", e.p1)
    print("P2:", e.p2)
    print()
    for v in vertices:
        if v != e.p1 and v != e.p2:
            print("Analyzing vertex:", v)
            ang = angle(sub(e.p1, v), sub(e.p2, v))
            print("The angle is:", ang)
            if ang > max_angle:
                max_angle = ang
                point = v
        else:
            print(v,"is part of edge.")
    
    if point is None:
        print("Object can't be triangulated.")
        break
    print("Choosen point:", point)        
    # Verificar se existe aresta e.p2->v e v->e.p1
    # Se existir, é a aresta procurada.
    # Se não, verificar se existe no sentido oposto, ou seja
    # v->e.p2 e e.p1->v
    # Se existir (?) inverter? Não sei ainda.
    # REF: https://github.com/carolhmj/advancing-front/blob/master/AdvancingFront/model.cpp
    # Se não existir, criar aresta.

First edge: (0.0, 0.0, 0.0) (0.0, 1.0, 0.0)
P1: (0.0, 0.0, 0.0)
P2: (0.0, 1.0, 0.0)

(0.0, 0.0, 0.0) is part of edge.
(0.0, 1.0, 0.0) is part of edge.
Analyzing vertex: (1.0, 0.0, 0.0)
The angle is: 0.7071067811865475
Analyzing vertex: (1.0, 1.0, 0.0)
The angle is: 0.7071067811865475
Choosen point: (1.0, 0.0, 0.0)
First edge: (0.0, 1.0, 0.0) (1.0, 0.0, 0.0)
P1: (0.0, 1.0, 0.0)
P2: (1.0, 0.0, 0.0)

Analyzing vertex: (0.0, 0.0, 0.0)
The angle is: 0.0
(0.0, 1.0, 0.0) is part of edge.
(1.0, 0.0, 0.0) is part of edge.
Analyzing vertex: (1.0, 1.0, 0.0)
The angle is: 0.0
Choosen point: (0.0, 0.0, 0.0)
First edge: (1.0, 0.0, 0.0) (1.0, 1.0, 0.0)
P1: (1.0, 0.0, 0.0)
P2: (1.0, 1.0, 0.0)

Analyzing vertex: (0.0, 0.0, 0.0)
The angle is: 0.7071067811865475
Analyzing vertex: (0.0, 1.0, 0.0)
The angle is: 0.7071067811865475
(1.0, 0.0, 0.0) is part of edge.
(1.0, 1.0, 0.0) is part of edge.
Choosen point: (0.0, 0.0, 0.0)
First edge: (1.0, 1.0, 0.0) (0.0, 0.0, 0.0)
P1: (1.0, 1.0, 0.0)
P2: (0.0, 0.0, 0.0